In [28]:
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-04-11 17:45:58--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.26
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.26|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-11 17:45:59--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1593 (1.6K) [text/plain]
Saving to: ‘STDOUT’

setup Colab for PySpark 3.1.1 and Spark NLP 3.0.1
-                   100%[===================>]   1.56K  --.-KB/s    in 0s      

2021-04-11 17:45:59 (48.7 M

In [56]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import lit
# from pyspark.sql.functions import desc
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.ml.recommendation import ALS
# from pyspark import SparkContext as sc
# from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf,col
import os
# tools
import re
import math
import json
import requests
import itertools
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
import string

In [82]:
# from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
# from pyspark.mllib.util import MLUtils
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#from pyspark.sql.functions import split as splitsp
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# %matplotlib inline
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# from pyspark.mllib.linalg import Vectors
# from pyspark.mllib.linalg.distributed import RowMatrix

In [31]:
'''
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark Naive Bayes") \
        .getOrCreate()
    return spark
spark = init_spark()
'''
import sparknlp
spark = sparknlp.start()
#spark = SparkSession.builder.appName("Spark NLP").master("local[4]").config("spark.driver.memory","16G").config("spark.driver.maxResultSize", "0").config("spark.kryoserializer.buffer.max", "2000M").config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.1").getOrCreate()

In [43]:
'''
Convert a percentage string into a float.
'''
def convert_percentage_nb(percentage):
  print(percentage)
  nb = float(percentage[:-1])

  return 1 if nb >= 6 else 2

In [115]:
'''
Give a class label to each post. 0 for >= 6% growth, otherwise 1.
'''
from pyspark.sql.types import IntegerType
data = spark.read.csv("100days-wsbdata.csv", header=True)

function = udf(lambda growth: 0 if float(growth[:-1]) >=6 else 1, IntegerType())
data = data.filter(data.growth != "N/A")
data = data.withColumn('label', function(data.growth))
print(data.count())
data.show()

1304
+------+------+-------+--------------------+----------+-----+------------+--------------------+------------+--------------------+----------+--------------------+----------+-----+
|    id|ticker| growth|               title|     flair|score|upvote_ratio|              author|num_comments|                text|   created|                 url| timestamp|label|
+------+------+-------+--------------------+----------+-----+------------+--------------------+------------+--------------------+----------+--------------------+----------+-----+
|krx451|   GME|811.69%|k gme sharesoptio...|      YOLO|   14|        0.94|  Hank-TheSpank-Hill|          42|     sharesoptions k|2021-01-06|https://www.reddi...|1609909200|    0|
|ks1tzw|   GME|811.69%|           gme🚀🚀🚀|Discussion|   28|        0.94|  PicksburghStillers|          37|all right all of ...|2021-01-06|https://www.reddi...|1609909200|    0|
|ksjhuu|  MGNI| 47.13%|$mgni 🌈🐻 have c...|        DD|    7|           1|     420No_Ragrets69|          

In [36]:
from sparknlp.pretrained import PretrainedPipeline
from sparknlp import Finisher
from pyspark.ml import Pipeline

In [116]:
finisher = Finisher().setInputCols(["token", "lemmas", "pos"])
explain_pipeline_model = PretrainedPipeline("explain_document_ml").model

pipeline = Pipeline() \
    .setStages([
        explain_pipeline_model,
        finisher
        ])

model = pipeline.fit(data.select('id', 'label', 'text'))
annotations_finished_df = model.transform(data.select('id','label', 'text'))

explain_document_ml download started this may take some time.
Approx size to download 9.1 MB
[OK!]


In [117]:
from pyspark.sql.types import BooleanType
text_lemmas = annotations_finished_df.select('id', 'label', 'finished_lemmas')
# remove posts for which the text is too short
function_length = udf(lambda r: len(r) > 10, BooleanType())
text_lemmas = text_lemmas.withColumn('word_count', function_length('finished_lemmas')).filter(col('word_count')).drop('word_count')
print("Total posts:", text_lemmas.count())
text_lemmas.show()

Total posts: 1144
+------+-----+--------------------+
|    id|label|     finished_lemmas|
+------+-----+--------------------+
|ks1tzw|    0|[all, right, all,...|
|ksjhuu|    0|[bldr, $, pt, pos...|
|kt79b2|    0|[we, might, be, h...|
|kt9enf|    0|[bear, feast, on,...|
|ktbu75|    0|[so, today, i, go...|
|ktfori|    0|[what, a, week, i...|
|kup2e2|    1|[start, to, feel,...|
|kurzyz|    0|[i, honestly, don...|
|kusng4|    0|[soi, recently, o...|
|kuku2g|    1|[nio, be, disrupt...|
|kutslm|    1|[listen, up, you,...|
|kv7k8k|    0|[have, anyone, no...|
|kvarzs|    0|[now, that, the, ...|
|kvcard|    0|[we, all, know, t...|
|kvcmhk|    1|[it, do, not, dil...|
|kva2kt|    0|[we, know, you, l...|
|kvacad|    1|[so, i, hear, you...|
|kvcimd|    1|[look, at, hour, ...|
|kvtk4b|    1|[giga, berlina, c...|
|kw2vwm|    1|[see, the, end, o...|
+------+-----+--------------------+
only showing top 20 rows



In [99]:
'''
Make lemmas writable to CSV
'''
# output = text_lemmas.rdd.map(lambda row: Row(id=row['id'], text='|'.join(row['finished_lemmas'])))
# output_df = output.toDF()
# output_df.show()

'\nMake lemmas writable to CSV\n'

In [40]:
'''
Write lemmas to CSV
'''
#output_df.coalesce(1).write.csv('lemmas-100daysdata.csv', header=True)

In [118]:
'''
Get the Corpus.
Removing stop words from the text lemmas. 
'''
remover = StopWordsRemover(inputCol="finished_lemmas", outputCol="text")
filtered_df = remover.transform(text_lemmas)
filtered_df.show()

+------+-----+--------------------+--------------------+
|    id|label|     finished_lemmas|                text|
+------+-----+--------------------+--------------------+
|ks1tzw|    0|[all, right, all,...|[right, artist, f...|
|ksjhuu|    0|[bldr, $, pt, pos...|[bldr, $, pt, pos...|
|kt79b2|    0|[we, might, be, h...|[might, hear, fir...|
|kt9enf|    0|[bear, feast, on,...|[bear, feast, low...|
|ktbu75|    0|[so, today, i, go...|[today, go, games...|
|ktfori|    0|[what, a, week, i...|[week, steel, big...|
|kup2e2|    1|[start, to, feel,...|[start, feel, rea...|
|kurzyz|    0|[i, honestly, don...|[honestly, dont, ...|
|kusng4|    0|[soi, recently, o...|[soi, recently, o...|
|kuku2g|    1|[nio, be, disrupt...|[nio, disrupt, te...|
|kutslm|    1|[listen, up, you,...|[listen, degenera...|
|kv7k8k|    0|[have, anyone, no...|[anyone, notice, ...|
|kvarzs|    0|[now, that, the, ...|[boy, rc, get, po...|
|kvcard|    0|[we, all, know, t...|[know, 🌈🐻s, cal...|
|kvcmhk|    1|[it, do, not, dil..

In [141]:
'''
[Skip this code cell if using HashingTF]
Create Document-Term Matrix by vectorizing the filtered text.
- returns the features column: (total nb of words, indices of each word in total vocab, count of each word)
'''
to_vectorize = filtered_df.select('id', 'label', 'text')
cv = CountVectorizer(inputCol="text", outputCol="features")
model_vec = cv.fit(to_vectorize)
result_vec = model_vec.transform(to_vectorize)
result.show()

+------+--------------------+--------------------+
|    id|                text|            features|
+------+--------------------+--------------------+
|krsa5c|[still, hold, maj...|(17391,[0,3,4,6,1...|
|krxpdq|[nio, day, come, ...|(17391,[3,12,16,1...|
|ks2don|[yeah, fucking, r...|(17391,[5,7,11,12...|
|ks3lno|[fellow, artist, ...|(17391,[1,2,7,15,...|
|ktbnq4|[weekend, discuss...|(17391,[70,177,44...|
|ktbvv1|[felon, chan, sup...|(17391,[0,1,2,4,5...|
|ktdjku|[exactly, fds, wa...|(17391,[6,14,24,2...|
|ktxy3u|[retarded, really...|(17391,[1,2,3,7,1...|
|kui6uh|[ive, follow, pos...|(17391,[1,6,8,10,...|
|kup2e2|[start, feel, rea...|(17391,[1,7,14,29...|
|kupo0b|[chaa, chai, mfer...|(17391,[2,3,7,9,1...|
|kurzyz|[honestly, dont, ...|(17391,[7,15,16,1...|
|kuu9xv|[ding, bft, new, ...|(17391,[0,12,18,1...|
|kv7k8k|[anyone, notice, ...|(17391,[0,4,26,27...|
|kv9s0l|[lucid, motor, sa...|(17391,[2,18,19,2...|
|kvanm4|[im, gonna, hones...|(17391,[1,5,10,11...|
|kvarzs|[boy, rc, get, po...|(1

In [66]:
'''
Join label onto vectors.
Total unique vocabulary count from CountVectorizer.
'''

#len(model_vec.vocabulary)

17390

In [142]:
'''
HashingTF
'''
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

hashingTF = HashingTF(inputCol="text", outputCol="rawFeatures", numFeatures=50)
featurizedData = hashingTF.transform(filtered_df)

featurizedData.show()

+------+-----+--------------------+--------------------+--------------------+
|    id|label|     finished_lemmas|                text|         rawFeatures|
+------+-----+--------------------+--------------------+--------------------+
|ks1tzw|    0|[all, right, all,...|[right, artist, f...|(50,[0,1,2,3,4,5,...|
|ksjhuu|    0|[bldr, $, pt, pos...|[bldr, $, pt, pos...|(50,[0,2,3,4,6,7,...|
|kt79b2|    0|[we, might, be, h...|[might, hear, fir...|(50,[3,4,7,14,21,...|
|kt9enf|    0|[bear, feast, on,...|[bear, feast, low...|(50,[0,1,3,5,6,7,...|
|ktbu75|    0|[so, today, i, go...|[today, go, games...|(50,[0,1,3,4,5,6,...|
|ktfori|    0|[what, a, week, i...|[week, steel, big...|(50,[0,1,3,4,5,6,...|
|kup2e2|    1|[start, to, feel,...|[start, feel, rea...|(50,[1,2,3,4,5,6,...|
|kurzyz|    0|[i, honestly, don...|[honestly, dont, ...|(50,[1,3,6,7,12,1...|
|kusng4|    0|[soi, recently, o...|[soi, recently, o...|(50,[1,4,5,6,7,10...|
|kuku2g|    1|[nio, be, disrupt...|[nio, disrupt, te...|(50,[0,1

In [143]:
'''
TF-IDF, followed from HashingTF.
'''
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [144]:
# convert label to numeric type
# function2 = udf(lambda r: float(r))
# rescaledData = rescaledData.withColumn('label', function2('label'))
# rescaledData.show()
selectedData = rescaledData.select('id', 'label','features', 'text')
selectedData.show()

+------+-----+--------------------+--------------------+
|    id|label|            features|                text|
+------+-----+--------------------+--------------------+
|ks1tzw|    0|(50,[0,1,2,3,4,5,...|[right, artist, f...|
|ksjhuu|    0|(50,[0,2,3,4,6,7,...|[bldr, $, pt, pos...|
|kt79b2|    0|(50,[3,7,11,14,21...|[might, hear, fir...|
|kt9enf|    0|(50,[0,1,3,5,6,7,...|[bear, feast, low...|
|ktbu75|    0|(50,[0,1,3,4,5,6,...|[today, go, games...|
|ktfori|    0|(50,[0,1,3,4,5,6,...|[week, steel, big...|
|kup2e2|    1|(50,[1,2,3,4,5,6,...|[start, feel, rea...|
|kurzyz|    0|(50,[1,3,6,7,12,1...|[honestly, dont, ...|
|kusng4|    0|(50,[1,4,5,6,7,10...|[soi, recently, o...|
|kuku2g|    1|(50,[0,1,2,3,4,5,...|[nio, disrupt, te...|
|kutslm|    1|(50,[0,1,2,4,5,6,...|[listen, degenera...|
|kv7k8k|    0|(50,[4,5,6,7,10,1...|[anyone, notice, ...|
|kvarzs|    0|(50,[1,3,4,6,7,8,...|[boy, rc, get, po...|
|kvcard|    0|(50,[0,1,2,3,4,5,...|[know, 🌈🐻s, cal...|
|kvcmhk|    1|(50,[3,4,6,7,8,9,..

In [145]:
'''
Separate data into training/test used for Naive-Bayes
'''
training_zero, test_zero = selectedData.where(selectedData.label == 0).randomSplit([0.7, 0.3])
training_one, test_one = selectedData.where(selectedData.label == 1).randomSplit([0.7, 0.3])

training = training_zero.union(training_one)
test = test_zero.union(test_one)
training.show()
# should be 70% of total in training, 30% in test
print("Total data count:", selectedData.count())
print("Total count of >6%", training.count())
print("Total count of <6%", test.count())

+------+-----+--------------------+--------------------+
|    id|label|            features|                text|
+------+-----+--------------------+--------------------+
|ks1tzw|    0|(50,[0,1,2,3,4,5,...|[right, artist, f...|
|ksjhuu|    0|(50,[0,2,3,4,6,7,...|[bldr, $, pt, pos...|
|kt79b2|    0|(50,[3,5,7,14,21,...|[might, hear, fir...|
|kt9enf|    0|(50,[0,1,3,5,6,7,...|[bear, feast, low...|
|ktbu75|    0|(50,[0,1,3,4,5,6,...|[today, go, games...|
|ktfori|    0|(50,[0,1,3,4,5,6,...|[week, steel, big...|
|kurzyz|    0|(50,[1,3,6,7,12,1...|[honestly, dont, ...|
|kv7k8k|    0|(50,[4,5,6,7,10,1...|[anyone, notice, ...|
|kvarzs|    0|(50,[1,3,4,6,7,8,...|[boy, rc, get, po...|
|kvcard|    0|(50,[0,1,2,3,4,5,...|[know, 🌈🐻s, cal...|
|kw6cgb|    0|(50,[0,1,3,4,11,1...|[gme, gang, look,...|
|kwjcqc|    0|(50,[0,1,2,4,5,6,...|[listen, fucker, ...|
|kwjyhm|    0|(50,[0,3,4,5,6,7,...|[youl, see, lot, ...|
|kwkf4t|    0|(50,[7,8,9,12,26,...|[idea, help, posi...|
|kwld39|    0|(50,[1,3,4,7,8,9,..

In [147]:
'''
Naive-Bayes following from TF-IDF
'''
print("Total data count:", selectedData.count())
print("Training data count:", training.count())
print("Testing data count:", test.count())
print("------------------------------------")
print("Total training count of >6%", training.where(selectedData.label == 0).count())
print("Total training count of <6%", training.where(selectedData.label == 1).count())
print("------------------------------------")
print("Total test count of >6%", test.where(selectedData.label == 0).count())
print("Total test count of <6%", test.where(selectedData.label == 1).count())

# create trainer with parameters then train
# smoothing: smooth probabilities of 0 to the input
nb = NaiveBayes(smoothing=0.3, modelType="complement")
model_NB = nb.fit(training)

# display on test set: appends a prediction column
predictions = model_NB.transform(test)
predictions.show()

# compute accuracy of on test set: compares labelCol and predictionCol
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print('Model accuracy:', accuracy)

Total data count: 1144
Training data count: 773
Testing data count: 371
------------------------------------
Total training count of >6% 377
Total training count of <6% 396
------------------------------------
Total test count of >6% 195
Total test count of <6% 176
+------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|    id|label|            features|                text|       rawPrediction|         probability|prediction|
+------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|kusng4|    0|(50,[1,4,5,6,7,10...|[soi, recently, o...|[-0.4828084719784...|[0.61704799403816...|       0.0|
|kva2kt|    0|(50,[0,2,3,4,5,6,...|[know, love, chew...|[-0.5426488512591...|[0.58120668152922...|       0.0|
|kw05rz|    0|(50,[0,6,7,10,11,...|[grab, data, part...|[-0.5773243439167...|[0.56139846801874...|       0.0|
|kwiti3|    0|(50,[1,3,5,6,7,10...|[paper, hand, bit...|[-0.5634137051895.